## بررسی خوشه‌بندی K-Means با استفاده از R و اصول داده‌های Tidy

### [**آزمون پیش از درس**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

در این درس، یاد می‌گیرید که چگونه با استفاده از بسته Tidymodels و دیگر بسته‌های موجود در اکوسیستم R (که به آن‌ها دوستان 🧑‍🤝‍🧑 می‌گوییم) و مجموعه داده‌های موسیقی نیجریه‌ای که قبلاً وارد کرده‌اید، خوشه‌ها ایجاد کنید. ما اصول اولیه K-Means برای خوشه‌بندی را پوشش خواهیم داد. به یاد داشته باشید که همان‌طور که در درس قبلی یاد گرفتید، روش‌های مختلفی برای کار با خوشه‌ها وجود دارد و روشی که استفاده می‌کنید به داده‌های شما بستگی دارد. ما K-Means را امتحان می‌کنیم زیرا رایج‌ترین تکنیک خوشه‌بندی است. بیایید شروع کنیم!

اصطلاحاتی که با آن‌ها آشنا خواهید شد:

-   امتیازدهی Silhouette

-   روش Elbow

-   اینرسی

-   واریانس

### **مقدمه**

[خوشه‌بندی K-Means](https://wikipedia.org/wiki/K-means_clustering) روشی است که از حوزه پردازش سیگنال مشتق شده است. این روش برای تقسیم و گروه‌بندی داده‌ها به `k خوشه` بر اساس شباهت ویژگی‌های آن‌ها استفاده می‌شود.

خوشه‌ها را می‌توان به صورت [نمودارهای ورونوی](https://wikipedia.org/wiki/Voronoi_diagram) تجسم کرد که شامل یک نقطه (یا 'بذر') و منطقه مربوط به آن است.

<p >
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>اینفوگرافیک از Jen Looper</figcaption>

مراحل خوشه‌بندی K-Means به شرح زیر است:

1.  دانشمند داده ابتدا تعداد خوشه‌های مورد نظر برای ایجاد را مشخص می‌کند.

2.  سپس الگوریتم به صورت تصادفی K مشاهده از مجموعه داده را به عنوان مراکز اولیه خوشه‌ها (یعنی centroidها) انتخاب می‌کند.

3.  در مرحله بعد، هر یک از مشاهدات باقی‌مانده به نزدیک‌ترین centroid اختصاص داده می‌شود.

4.  سپس میانگین جدید هر خوشه محاسبه شده و centroid به میانگین منتقل می‌شود.

5.  اکنون که مراکز دوباره محاسبه شده‌اند، هر مشاهده دوباره بررسی می‌شود تا مشخص شود آیا ممکن است به خوشه دیگری نزدیک‌تر باشد. تمام اشیاء دوباره با استفاده از میانگین‌های به‌روزرسانی‌شده خوشه‌ها تخصیص داده می‌شوند. مراحل تخصیص خوشه و به‌روزرسانی centroid به صورت تکراری انجام می‌شود تا زمانی که تخصیص خوشه‌ها تغییر نکند (یعنی زمانی که همگرایی حاصل شود). معمولاً الگوریتم زمانی متوقف می‌شود که هر تکرار جدید منجر به حرکت ناچیز centroidها شود و خوشه‌ها ثابت شوند.

<div>

> توجه داشته باشید که به دلیل تصادفی بودن انتخاب اولیه k مشاهده به عنوان centroidهای شروع، ممکن است هر بار که این روش را اعمال می‌کنیم نتایج کمی متفاوت باشد. به همین دلیل، اکثر الگوریتم‌ها از چندین *شروع تصادفی* استفاده می‌کنند و تکراری را که کمترین WCSS را دارد انتخاب می‌کنند. بنابراین، اکیداً توصیه می‌شود که همیشه K-Means را با چندین مقدار *nstart* اجرا کنید تا از یک *بهینه محلی نامطلوب* جلوگیری شود.

</div>

این انیمیشن کوتاه با استفاده از [آثار هنری](https://github.com/allisonhorst/stats-illustrations) از Allison Horst فرآیند خوشه‌بندی را توضیح می‌دهد:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>اثر هنری از @allison_horst</figcaption>

یک سؤال اساسی که در خوشه‌بندی مطرح می‌شود این است: چگونه می‌دانید که داده‌های خود را به چند خوشه تقسیم کنید؟ یکی از معایب استفاده از K-Means این است که باید `k`، یعنی تعداد `centroidها` را تعیین کنید. خوشبختانه، `روش Elbow` به تخمین یک مقدار اولیه مناسب برای `k` کمک می‌کند. به زودی آن را امتحان خواهید کرد.

### 

**پیش‌نیاز**

ما از جایی که در [درس قبلی](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb) متوقف شدیم، ادامه می‌دهیم، جایی که مجموعه داده را تحلیل کردیم، تعداد زیادی تجسم ایجاد کردیم و مجموعه داده را به مشاهدات مورد علاقه فیلتر کردیم. حتماً آن را بررسی کنید!

برای این ماژول به چند بسته نیاز داریم. می‌توانید آن‌ها را با دستور زیر نصب کنید:  
`install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

یا از اسکریپت زیر استفاده کنید که بررسی می‌کند آیا بسته‌های مورد نیاز برای تکمیل این ماژول را دارید و در صورت کمبود، آن‌ها را برای شما نصب می‌کند.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


بیایید سریع شروع کنیم!

## ۱. رقصی با داده‌ها: محدود کردن به ۳ سبک موسیقی محبوب

این یک مرور کلی از کاری است که در درس قبلی انجام دادیم. بیایید کمی داده‌ها را بررسی و تحلیل کنیم!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 این خیلی خوب پیش رفت!

## ۲. بررسی بیشتر داده‌ها

این داده‌ها چقدر تمیز هستند؟ بیایید با استفاده از نمودارهای جعبه‌ای به دنبال نقاط پرت بگردیم. ما بر روی ستون‌های عددی با نقاط پرت کمتر تمرکز خواهیم کرد (اگرچه می‌توانید نقاط پرت را حذف کنید). نمودارهای جعبه‌ای می‌توانند محدوده داده‌ها را نشان دهند و به انتخاب ستون‌های مناسب کمک کنند. توجه داشته باشید که نمودارهای جعبه‌ای واریانس را نشان نمی‌دهند، که یک عنصر مهم برای داده‌های خوشه‌بندی‌شدنی خوب است. لطفاً برای مطالعه بیشتر به [این بحث](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot) مراجعه کنید.

[نمودارهای جعبه‌ای](https://en.wikipedia.org/wiki/Box_plot) برای نمایش گرافیکی توزیع داده‌های `عددی` استفاده می‌شوند، بنابراین بیایید با *انتخاب* تمام ستون‌های عددی در کنار ژانرهای محبوب موسیقی شروع کنیم.


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


ببینید چطور ابزار انتخاب `where` این کار را ساده می‌کند 💁؟ سایر توابع مشابه را [اینجا](https://tidyselect.r-lib.org/) بررسی کنید.

از آنجایی که قصد داریم برای هر ویژگی عددی یک نمودار جعبه‌ای بسازیم و می‌خواهیم از استفاده از حلقه‌ها اجتناب کنیم، بهتر است داده‌های خود را به یک قالب *طولانی‌تر* تغییر دهیم که به ما امکان استفاده از `facets` را بدهد - زیرنمودارهایی که هر کدام یک زیرمجموعه از داده‌ها را نمایش می‌دهند.


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


حالا وقتشه که با `ggplot`ها بیشتر کار کنیم! پس از کدوم `geom` استفاده خواهیم کرد؟


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


حالا می‌بینیم که این داده‌ها کمی پر سر و صدا هستند: با مشاهده هر ستون به صورت یک نمودار جعبه‌ای، می‌توانید نقاط پرت را ببینید. می‌توانید این نقاط پرت را از مجموعه داده حذف کنید، اما این کار داده‌ها را خیلی محدود می‌کند.

فعلاً بیایید انتخاب کنیم که کدام ستون‌ها را برای تمرین خوشه‌بندی خود استفاده کنیم. بیایید ستون‌های عددی با بازه‌های مشابه را انتخاب کنیم. می‌توانستیم `artist_top_genre` را به صورت عددی کدگذاری کنیم، اما فعلاً آن را کنار می‌گذاریم.


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## ۳. محاسبه خوشه‌بندی k-means در R

می‌توانیم خوشه‌بندی k-means را در R با استفاده از تابع داخلی `kmeans` محاسبه کنیم. برای اطلاعات بیشتر می‌توانید از `help("kmeans()")` استفاده کنید. تابع `kmeans()` یک داده‌فریم با ستون‌های عددی را به عنوان آرگومان اصلی خود می‌پذیرد.

اولین گام در استفاده از خوشه‌بندی k-means مشخص کردن تعداد خوشه‌ها (k) است که در نهایت ایجاد خواهند شد. می‌دانیم که در مجموعه داده ما سه ژانر موسیقی وجود دارد، پس بیایید عدد ۳ را امتحان کنیم:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


شیء kmeans شامل اطلاعات مختلفی است که در `help("kmeans()")` به خوبی توضیح داده شده است. فعلاً، بیایید روی چند مورد تمرکز کنیم. مشاهده می‌کنیم که داده‌ها به ۳ خوشه با اندازه‌های ۶۵، ۱۱۰ و ۱۱۱ گروه‌بندی شده‌اند. خروجی همچنین شامل مراکز خوشه‌ها (میانگین‌ها) برای ۳ گروه در میان ۵ متغیر است.

بردار خوشه‌بندی، تخصیص خوشه برای هر مشاهده است. بیایید از تابع `augment` استفاده کنیم تا تخصیص خوشه را به مجموعه داده اصلی اضافه کنیم.


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


ما به‌تازگی مجموعه داده خود را به سه گروه تقسیم کردیم. حالا، خوشه‌بندی ما چقدر خوب است 🤷؟ بیایید نگاهی به امتیاز `Silhouette` بیندازیم.

### **امتیاز Silhouette**

[تحلیل Silhouette](https://en.wikipedia.org/wiki/Silhouette_(clustering)) می‌تواند برای بررسی فاصله جدایی بین خوشه‌های حاصل استفاده شود. این امتیاز از -1 تا 1 متغیر است، و اگر امتیاز نزدیک به 1 باشد، خوشه متراکم و به‌خوبی از سایر خوشه‌ها جدا شده است. مقدار نزدیک به 0 نشان‌دهنده خوشه‌های هم‌پوشان با نمونه‌هایی است که بسیار نزدیک به مرز تصمیم‌گیری خوشه‌های همسایه هستند. [منبع](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam).

روش میانگین Silhouette میانگین امتیاز Silhouette مشاهدات را برای مقادیر مختلف *k* محاسبه می‌کند. امتیاز میانگین Silhouette بالا نشان‌دهنده خوشه‌بندی خوب است.

تابع `silhouette` در بسته خوشه‌بندی برای محاسبه میانگین عرض Silhouette استفاده می‌شود.

> Silhouette را می‌توان با هر [فاصله](https://en.wikipedia.org/wiki/Distance "Distance") متریک محاسبه کرد، مانند [فاصله اقلیدسی](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") یا [فاصله منهتن](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance") که در [درس قبلی](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb) درباره آن صحبت کردیم.


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


امتیاز ما **.549** است، یعنی دقیقاً در حد وسط. این نشان می‌دهد که داده‌های ما برای این نوع خوشه‌بندی خیلی مناسب نیستند. بیایید ببینیم آیا می‌توانیم این حدس را به صورت بصری تأیید کنیم. [بسته factoextra](https://rpkgs.datanovia.com/factoextra/index.html) توابعی مانند (`fviz_cluster()`) را برای بصری‌سازی خوشه‌بندی ارائه می‌دهد.


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


همپوشانی در خوشه‌ها نشان می‌دهد که داده‌های ما برای این نوع خوشه‌بندی چندان مناسب نیست، اما بیایید ادامه دهیم.

## ۴. تعیین تعداد بهینه خوشه‌ها

یک سؤال اساسی که اغلب در خوشه‌بندی K-Means مطرح می‌شود این است که - بدون داشتن برچسب‌های کلاس مشخص، چگونه می‌توان فهمید که داده‌ها را باید به چند خوشه تقسیم کرد؟

یکی از روش‌هایی که می‌توانیم امتحان کنیم این است که از یک نمونه داده استفاده کرده و `یک سری مدل خوشه‌بندی` با تعداد خوشه‌های افزایشی (مثلاً از ۱ تا ۱۰) ایجاد کنیم و معیارهای خوشه‌بندی مانند **امتیاز Silhouette** را ارزیابی کنیم.

بیایید تعداد بهینه خوشه‌ها را با اجرای الگوریتم خوشه‌بندی برای مقادیر مختلف *k* و ارزیابی **مجموع مربعات درون خوشه‌ای** (WCSS) تعیین کنیم. مجموع مربعات درون خوشه‌ای (WCSS) میزان فشردگی خوشه‌بندی را اندازه‌گیری می‌کند و ما می‌خواهیم این مقدار تا حد ممکن کوچک باشد، زیرا مقادیر کمتر نشان می‌دهند که نقاط داده به هم نزدیک‌تر هستند.

بیایید تأثیر انتخاب‌های مختلف `k`، از ۱ تا ۱۰، را بر این خوشه‌بندی بررسی کنیم.


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


اکنون که مجموع کل مربعات درون خوشه‌ها (tot.withinss) را برای هر الگوریتم خوشه‌بندی با مرکز *k* داریم، از [روش آرنج](https://en.wikipedia.org/wiki/Elbow_method_(clustering)) برای یافتن تعداد بهینه خوشه‌ها استفاده می‌کنیم. این روش شامل رسم WCSS به عنوان تابعی از تعداد خوشه‌ها و انتخاب [نقطه آرنج منحنی](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Elbow of the curve") به عنوان تعداد خوشه‌های مورد استفاده است.


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


نمودار کاهش قابل توجهی در WCSS (بنابراین *فشردگی* بیشتر) را نشان می‌دهد وقتی تعداد خوشه‌ها از یک به دو افزایش می‌یابد، و کاهش قابل توجه دیگری از دو به سه خوشه مشاهده می‌شود. پس از آن، کاهش کمتر محسوس است و در نمودار یک `خمیدگی` 💪 در حدود سه خوشه ایجاد می‌شود. این نشان‌دهنده خوبی است که دو تا سه خوشه به طور منطقی از نقاط داده جدا شده‌اند.

اکنون می‌توانیم مدل خوشه‌بندی را که در آن `k = 3` است استخراج کنیم:

> `pull()`: برای استخراج یک ستون استفاده می‌شود
>
> `pluck()`: برای ایندکس کردن ساختارهای داده‌ای مانند لیست‌ها استفاده می‌شود


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


عالی! بیایید خوشه‌های به‌دست‌آمده را تجسم کنیم. علاقه‌مند به کمی تعامل با استفاده از `plotly` هستید؟


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


شاید انتظار داشتیم که هر خوشه (نمایش داده شده با رنگ‌های مختلف) ژانرهای متمایزی (نمایش داده شده با اشکال مختلف) داشته باشد.

بیایید نگاهی به دقت مدل بیندازیم.


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


دقت این مدل بد نیست، اما عالی هم نیست. ممکن است داده‌ها برای خوشه‌بندی K-Means مناسب نباشند. این داده‌ها بسیار نامتوازن هستند، همبستگی کمی دارند و تنوع زیادی بین مقادیر ستون‌ها وجود دارد که باعث می‌شود خوشه‌بندی به خوبی انجام نشود. در واقع، خوشه‌هایی که تشکیل می‌شوند احتمالاً به شدت تحت تأثیر یا منحرف شده توسط سه دسته ژانری هستند که قبلاً تعریف کردیم.

با این حال، این یک فرآیند یادگیری جالب بود!

در مستندات Scikit-learn، می‌توانید ببینید که مدلی مانند این، با خوشه‌هایی که به خوبی مشخص نشده‌اند، یک مشکل «واریانس» دارد:

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>اینفوگرافیک از Scikit-learn</figcaption>



## **واریانس**

واریانس به عنوان "میانگین مربع تفاوت‌ها از میانگین" تعریف می‌شود [منبع](https://www.mathsisfun.com/data/standard-deviation.html). در زمینه این مسئله خوشه‌بندی، به داده‌هایی اشاره دارد که اعداد مجموعه داده ما تمایل دارند کمی بیش از حد از میانگین فاصله بگیرند.

✅ این یک لحظه عالی است برای فکر کردن به تمام راه‌هایی که می‌توانید این مشکل را اصلاح کنید. کمی داده‌ها را تغییر دهید؟ از ستون‌های مختلف استفاده کنید؟ از الگوریتم دیگری استفاده کنید؟ نکته: [مقیاس‌بندی داده‌ها](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/) را امتحان کنید تا آن‌ها را نرمال کنید و ستون‌های دیگر را آزمایش کنید.

> از این '[ماشین حساب واریانس](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)' استفاده کنید تا مفهوم را بهتر درک کنید.

------------------------------------------------------------------------

## **🚀چالش**

زمانی را با این دفترچه صرف کنید و پارامترها را تغییر دهید. آیا می‌توانید با پاکسازی بیشتر داده‌ها (مثلاً حذف نقاط پرت) دقت مدل را بهبود دهید؟ می‌توانید از وزن‌ها برای دادن وزن بیشتر به نمونه‌های خاص داده استفاده کنید. چه کار دیگری می‌توانید انجام دهید تا خوشه‌های بهتری ایجاد کنید؟

نکته: مقیاس‌بندی داده‌ها را امتحان کنید. کدی که در دفترچه کامنت شده است، مقیاس‌بندی استاندارد را اضافه می‌کند تا ستون‌های داده از نظر محدوده به یکدیگر نزدیک‌تر شوند. خواهید دید که در حالی که امتیاز سیلوئت کاهش می‌یابد، «خمیدگی» در نمودار آرنج صاف‌تر می‌شود. این به این دلیل است که باقی گذاشتن داده‌ها بدون مقیاس‌بندی به داده‌هایی با واریانس کمتر اجازه می‌دهد وزن بیشتری داشته باشند. کمی بیشتر در مورد این مشکل [اینجا](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226) بخوانید.

## [**آزمون پس از درس**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **مرور و مطالعه شخصی**

-   به یک شبیه‌ساز K-Means [مانند این](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/) نگاهی بیندازید. می‌توانید از این ابزار برای تجسم نقاط داده نمونه و تعیین مراکز آن‌ها استفاده کنید. می‌توانید تصادفی بودن داده‌ها، تعداد خوشه‌ها و تعداد مراکز را ویرایش کنید. آیا این به شما کمک می‌کند ایده‌ای از نحوه گروه‌بندی داده‌ها بگیرید؟

-   همچنین، به [این جزوه درباره K-Means](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) از دانشگاه استنفورد نگاهی بیندازید.

آیا می‌خواهید مهارت‌های خوشه‌بندی تازه کسب‌شده خود را روی مجموعه داده‌هایی که برای خوشه‌بندی K-Means مناسب هستند امتحان کنید؟ لطفاً ببینید:

-   [آموزش و ارزیابی مدل‌های خوشه‌بندی](https://rpubs.com/eR_ic/clustering) با استفاده از Tidymodels و دوستان

-   [تحلیل خوشه‌بندی K-Means](https://uc-r.github.io/kmeans_clustering)، راهنمای برنامه‌نویسی R برای تحلیل کسب‌وکار UC

- [خوشه‌بندی K-Means با اصول داده‌های مرتب](https://www.tidymodels.org/learn/statistics/k-means/)

## **تکلیف**

[روش‌های مختلف خوشه‌بندی را امتحان کنید](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## سپاسگزاریم از:

[Jen Looper](https://www.twitter.com/jenlooper) برای ایجاد نسخه اصلی پایتون این ماژول ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) برای خلق تصاویر شگفت‌انگیزی که R را جذاب‌تر و دوستانه‌تر می‌کنند. تصاویر بیشتر را در [گالری او](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM) پیدا کنید.

یادگیری خوشایند،

[Eric](https://twitter.com/ericntay)، سفیر دانشجوی طلایی Microsoft Learn.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>اثر هنری از @allison_horst</figcaption>



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما برای دقت تلاش می‌کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادقتی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، ترجمه حرفه‌ای انسانی توصیه می‌شود. ما هیچ مسئولیتی در قبال سوءتفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
